In [1]:
import pymssql
import pandas as pd

#取得store_info_id
with pymssql.connect(host='20.3.171.50',user='sa',password='ntub_pj123456',database='food') as conn:
    sql = "SELECT store_info_id, store_name FROM store_info"
    store_info_id = pd.read_sql(sql,conn)

store_info_id

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,store_info_id,store_name
0,1,黃龍莊
1,2,悄悄好食 杭州南店
2,3,阿鸞手工法國麵包
3,4,有煎餃子館(北市寧波館)
4,5,張吳記麵餅舖
...,...,...
952,953,水源福利會館
953,954,飯饌韓式料理
954,955,豪季水餃專賣店 牯嶺店
955,956,小日子商號


In [2]:
for store_info_id in store_info_id["store_info_id"]:
    with pymssql.connect(host='20.3.171.50',user='sa',password='ntub_pj123456',database='food') as conn:
        
        #取得本地分數
        sql = "SELECT CONVERT(varchar, create_time, 111) AS create_time, score FROM store_local_comment WHERE (store_info_id = {}) AND (score <> 999) ORDER BY create_time".format(store_info_id)
        local_score = pd.read_sql(sql,conn)

        #取得外部分數
        sql = "SELECT CONVERT(varchar, create_time, 111) AS create_time, score FROM store_external_comment WHERE (store_info_id = {}) AND (score <> 999) ORDER BY create_time".format(store_info_id)
        external_score = pd.read_sql(sql,conn)
        
        #合併 本地分數 + 外部分數
        all_score = pd.concat([local_score, external_score], axis=0)
        print("店家id：" + str(store_info_id) + "，共" + str(len(all_score)) + "筆分數")
        
        #取月份
        all_score["create_time"] = all_score["create_time"].apply(lambda x:x[:7])
        
        #依月份分組
        all_score = all_score.groupby("create_time")["score"].mean().to_frame()
        
        #print(all_score)
        
        #存成(年月:分數)的一串文字
        score_record = ""
        for index, row in all_score.iterrows():
            date = index
            score = round(row["score"], 1)
            
            score_record += date + ":" + str(score) + ","
        score_record = score_record[:-1]
        
        print(score_record)
        
        #存入資料庫
        with conn.cursor(as_dict=True) as cursor:
            sql = "UPDATE store_info SET total_score_record = N'{}' WHERE (store_info_id = {})".format(score_record, store_info_id)
            cursor.execute(sql)
            conn.commit()        

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

店家id：1，共7筆分數
2017/04:4.0,2018/11:5.0,2019/03:2.0,2020/04:3.0,2022/06:4.5,2022/10:4.5
店家id：2，共12筆分數
2018/04:4.5,2018/06:4.5,2018/07:3.5,2018/08:5.0,2019/01:3.0,2020/11:4.5,2021/01:5.0,2021/05:3.0,2021/09:4.5,2022/04:4.5,2022/06:4.5,2022/10:5.0
店家id：3，共1筆分數
2021/08:3.0
店家id：4，共3筆分數
2020/08:4.0,2022/10:4.0
店家id：5，共2筆分數
2018/09:4.5,2022/03:3.5


C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

店家id：6，共0筆分數

店家id：7，共1筆分數
2022/11:0.0
店家id：8，共6筆分數
2016/10:4.0,2019/05:4.5,2020/05:4.0,2021/10:3.5,2022/08:4.5,2022/09:5.0
店家id：9，共2筆分數
2019/08:4.5,2022/10:5.0
店家id：10，共1筆分數
2022/10:5.0

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

...

C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

店家id：944，共1筆分數
2019/06:5.0
店家id：945，共0筆分數

店家id：946，共1筆分數
2019/06:3.5
店家id：947，共1筆分數
2019/05:4.5
店家id：948，共3筆分數
2017/07:4.0,2019/05:5.0,2019/11:1.0
店家id：949，共1筆分數
2017/09:5.0
店家id：950，共1筆分數
2019/05:4.5
店家id：951，共0筆分數

店家id：952，共1筆分數
2019/05:3.5
店家id：953，共1筆分數
2019/05:4.5
店家id：954，共5筆分數
2017/03:4.0,2017/05:3.5,2018/12:3.0,2019/05:4.0,2020/05:2.0
店家id：955，共0筆分數



C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI

店家id：956，共5筆分數
2017/02:3.5,2017/04:5.0,2017/07:4.8,2019/05:5.0
店家id：957，共0筆分數



C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\ntub_pj\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI